**This Notebook provides an ensemble model that combines the inference output from EffecientNet model and 2.5D s-stage model.**

## Load Package

In [222]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import cv2
import os
from os import listdir
import re
import gc
from time import time
from datetime import datetime
from path import Path
from sklearn.model_selection import train_test_split
import random
import pandas as pd
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    fbeta_score,
    recall_score,
    precision_score,
)


import warnings
warnings.filterwarnings('ignore')

## Import inference output

#### 2-stage model output

In [7]:
two_stage = pd.read_csv('valid_folds_all.csv')
two_stage['prediction_type'] = 'C'+two_stage['cid'].astype(str)
two_stage

,study_cid,StudyInstanceUID,cid,slice_num_list,before_image_size,x0,x1,y0,y1,z0,z1,label,Preds,Trues,prediction_type
0,1.2.826.0.1.3680043.10001_1,1.2.826.0.1.3680043.10001,1,"[50, 51, 52, 53, 55, 56, 57, 59, 60, 61, 63, 6...",320,0,267,2,273,48,277,0,0.113253,0,C1
1,1.2.826.0.1.3680043.10001_2,1.2.826.0.1.3680043.10001,2,"[1, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92...",320,0,267,2,273,48,277,0,0.192726,0,C2
2,1.2.826.0.1.3680043.10001_3,1.2.826.0.1.3680043.10001,3,"[108, 109, 110, 111, 112, 113, 114, 115, 116, ...",320,0,267,2,273,48,277,0,0.092990,0,C3
3,1.2.826.0.1.3680043.10001_4,1.2.826.0.1.3680043.10001,4,"[129, 130, 131, 132, 133, 134, 135, 136, 137, ...",320,0,267,2,273,48,277,0,0.079985,0,C4
4,1.2.826.0.1.3680043.10001_5,1.2.826.0.1.3680043.10001,5,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ...",320,0,267,2,273,48,277,0,0.123712,0,C5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13555,1.2.826.0.1.3680043.9997_3,1.2.826.0.1.3680043.9997,3,"[93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 10...",320,0,254,2,262,3,252,0,0.166747,0,C3
13556,1.2.826.0.1.3680043.9997_4,1.2.826.0.1.3680043.9997,4,"[114, 115, 116, 117, 118, 119, 120, 121, 122, ...",320,0,254,2,262,3,252,0,0.150317,0,C4
13557,1.2.826.0.1.3680043.9997_5,1.2.826.0.1.3680043.9997,5,"[136, 137, 138, 139, 140, 141, 142, 144, 145, ...",320,0,254,2,262,3,252,0,0.377570,0,C5
13558,1.2.826.0.1.3680043.9997_6,1.2.826.0.1.3680043.9997,6,"[158, 159, 160, 161, 162, 163, 164, 165, 166, ...",320,0,254,2,262,3,252,0,0.838126,0,C6


#### EfficientNet Output

In [3]:
effnet = pd.read_csv('effent_inference_500_patients.csv')
effnet

,actual,row_id,StudyInstanceUID,prediction_type,fractured,inference_time,pred_flag,Accuracy
0,1,1.2.826.0.1.3680043.6200_patient_overall,1.2.826.0.1.3680043.6200,patient_overall,0.132991,12.565742,0,0
1,1,1.2.826.0.1.3680043.6200_C1,1.2.826.0.1.3680043.6200,C1,0.315373,12.764569,0,0
2,1,1.2.826.0.1.3680043.6200_C2,1.2.826.0.1.3680043.6200,C2,0.423518,10.976760,1,1
3,0,1.2.826.0.1.3680043.6200_C3,1.2.826.0.1.3680043.6200,C3,0.095354,14.120935,0,1
4,0,1.2.826.0.1.3680043.6200_C4,1.2.826.0.1.3680043.6200,C4,0.333067,10.120789,0,1
...,...,...,...,...,...,...,...,...
3995,0,1.2.826.0.1.3680043.14654_C3,1.2.826.0.1.3680043.14654,C3,0.348316,42.737215,1,0
3996,0,1.2.826.0.1.3680043.14654_C4,1.2.826.0.1.3680043.14654,C4,0.296459,42.737215,0,1
3997,0,1.2.826.0.1.3680043.14654_C5,1.2.826.0.1.3680043.14654,C5,0.786132,42.737215,1,0
3998,0,1.2.826.0.1.3680043.14654_C6,1.2.826.0.1.3680043.14654,C6,0.186581,42.737215,0,1


### Combines output of 2 models

In [135]:
overall_infer = two_stage.merge(effnet, how='inner',
                                left_on = ['StudyInstanceUID','prediction_type'], 
                                right_on = ['StudyInstanceUID','prediction_type'])
overall_infer['two_stage_pred'] = overall_infer['Preds']
overall_infer['effnet_pred'] = overall_infer['fractured']
overall_infer = overall_infer[['StudyInstanceUID','prediction_type','cid','x0','x1',
                               'y0','y1','z0','z1','label','two_stage_pred','effnet_pred']]

overall_infer

,StudyInstanceUID,prediction_type,cid,x0,x1,y0,y1,z0,z1,label,two_stage_pred,effnet_pred
0,1.2.826.0.1.3680043.10032,C1,1,0,320,1,314,72,243,0,0.114832,0.691892
1,1.2.826.0.1.3680043.10032,C2,2,0,320,1,314,72,243,0,0.192151,0.412402
2,1.2.826.0.1.3680043.10032,C3,3,0,320,1,314,72,243,0,0.083235,0.345822
3,1.2.826.0.1.3680043.10032,C4,4,0,320,1,314,72,243,0,0.153492,0.384723
4,1.2.826.0.1.3680043.10032,C5,5,0,320,1,314,72,243,0,0.175922,0.816983
...,...,...,...,...,...,...,...,...,...,...,...,...
3357,1.2.826.0.1.3680043.9996,C3,3,0,295,3,320,72,254,0,0.083757,0.114156
3358,1.2.826.0.1.3680043.9996,C4,4,0,295,3,320,72,254,0,0.129619,0.344138
3359,1.2.826.0.1.3680043.9996,C5,5,0,295,3,320,72,254,0,0.117988,0.473295
3360,1.2.826.0.1.3680043.9996,C6,6,0,295,3,320,72,254,0,0.319698,0.275896


#### Add the previous calculated classification threshold

In [136]:
threshold = {'prediction_type': ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'],
             'two_stage_threshold': [0.23, 0.34, 0.23, 0.15, 0.19, 0.23, 0.42],
             'effnet_threshold': [0.67, 0.42, 0.31, 0.34, 0.73, 0.21, 0.81],
            }
  

two_stage_threshold = pd.DataFrame(threshold)
overall_infer = overall_infer.merge(two_stage_threshold, how='inner',
                                left_on = 'prediction_type', 
                                right_on = 'prediction_type')
overall_infer

,StudyInstanceUID,prediction_type,cid,x0,x1,y0,y1,z0,z1,label,two_stage_pred,effnet_pred,two_stage_threshold,effnet_threshold
0,1.2.826.0.1.3680043.10032,C1,1,0,320,1,314,72,243,0,0.114832,0.691892,0.23,0.67
1,1.2.826.0.1.3680043.10062,C1,1,0,349,1,316,2,262,0,0.091811,0.332340,0.23,0.67
2,1.2.826.0.1.3680043.10136,C1,1,0,357,2,317,76,250,0,0.124926,0.695181,0.23,0.67
3,1.2.826.0.1.3680043.10412,C1,1,0,684,2,216,69,230,0,0.120831,0.685851,0.23,0.67
4,1.2.826.0.1.3680043.10443,C1,1,0,601,2,247,84,248,0,0.218730,0.683365,0.23,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3357,1.2.826.0.1.3680043.9809,C7,7,0,568,3,224,82,237,0,0.388187,0.791127,0.42,0.81
3358,1.2.826.0.1.3680043.9921,C7,7,0,261,0,317,2,314,1,0.536955,0.847256,0.42,0.81
3359,1.2.826.0.1.3680043.9940,C7,7,41,257,40,315,67,281,0,0.437208,0.195100,0.42,0.81
3360,1.2.826.0.1.3680043.9994,C7,7,0,264,5,320,89,238,0,0.613176,0.811858,0.42,0.81


#### Add patient overall rows

In [132]:
patient_id = overall_infer['StudyInstanceUID'].drop_duplicates()
train_id = np.random.choice(patient_id,400, replace=False)

In [133]:
train_label = {'StudyInstanceUID': train_id,
             'train': [1]*len(train_id)
            }
train = pd.DataFrame(train_label)

,StudyInstanceUID,train
0,1.2.826.0.1.3680043.1381,1
1,1.2.826.0.1.3680043.4270,1
2,1.2.826.0.1.3680043.12666,1
3,1.2.826.0.1.3680043.29351,1
4,1.2.826.0.1.3680043.20038,1
...,...,...
395,1.2.826.0.1.3680043.21350,1
396,1.2.826.0.1.3680043.28968,1
397,1.2.826.0.1.3680043.3130,1
398,1.2.826.0.1.3680043.9548,1


In [137]:
overall_infer = overall_infer.merge(train, how='left',
                                left_on = 'StudyInstanceUID', 
                                right_on = 'StudyInstanceUID').drop_duplicates()
overall_infer.loc[overall_infer["train"] != 1, "train"] = 0
overall_infer

,StudyInstanceUID,prediction_type,cid,x0,x1,y0,y1,z0,z1,label,two_stage_pred,effnet_pred,two_stage_threshold,effnet_threshold,train
0,1.2.826.0.1.3680043.10032,C1,1,0,320,1,314,72,243,0,0.114832,0.691892,0.23,0.67,1.0
1,1.2.826.0.1.3680043.10062,C1,1,0,349,1,316,2,262,0,0.091811,0.332340,0.23,0.67,1.0
2,1.2.826.0.1.3680043.10136,C1,1,0,357,2,317,76,250,0,0.124926,0.695181,0.23,0.67,1.0
3,1.2.826.0.1.3680043.10412,C1,1,0,684,2,216,69,230,0,0.120831,0.685851,0.23,0.67,1.0
4,1.2.826.0.1.3680043.10443,C1,1,0,601,2,247,84,248,0,0.218730,0.683365,0.23,0.67,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3357,1.2.826.0.1.3680043.9809,C7,7,0,568,3,224,82,237,0,0.388187,0.791127,0.42,0.81,1.0
3358,1.2.826.0.1.3680043.9921,C7,7,0,261,0,317,2,314,1,0.536955,0.847256,0.42,0.81,0.0
3359,1.2.826.0.1.3680043.9940,C7,7,41,257,40,315,67,281,0,0.437208,0.195100,0.42,0.81,1.0
3360,1.2.826.0.1.3680043.9994,C7,7,0,264,5,320,89,238,0,0.613176,0.811858,0.42,0.81,1.0


#### Save combined inference

In [138]:
overall_infer.to_csv('overall_infer.csv', index=False)

## Train Test Split

In [125]:
overall_infer = pd.read_csv('overall_infer.csv')

,StudyInstanceUID,prediction_type,cid,x0,x1,y0,y1,z0,z1,label,two_stage_pred,effnet_pred,two_stage_threshold,effnet_threshold,train
0,1.2.826.0.1.3680043.10032,C1,1,0,320,1,314,72,243,0,0.114832,0.691892,0.23,0.67,0.0
1,1.2.826.0.1.3680043.10062,C1,1,0,349,1,316,2,262,0,0.091811,0.332340,0.23,0.67,1.0
2,1.2.826.0.1.3680043.10136,C1,1,0,357,2,317,76,250,0,0.124926,0.695181,0.23,0.67,0.0
3,1.2.826.0.1.3680043.10412,C1,1,0,684,2,216,69,230,0,0.120831,0.685851,0.23,0.67,1.0
4,1.2.826.0.1.3680043.10443,C1,1,0,601,2,247,84,248,0,0.218730,0.683365,0.23,0.67,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3357,1.2.826.0.1.3680043.9809,C7,7,0,568,3,224,82,237,0,0.388187,0.791127,0.42,0.81,1.0
3358,1.2.826.0.1.3680043.9921,C7,7,0,261,0,317,2,314,1,0.536955,0.847256,0.42,0.81,1.0
3359,1.2.826.0.1.3680043.9940,C7,7,41,257,40,315,67,281,0,0.437208,0.195100,0.42,0.81,1.0
3360,1.2.826.0.1.3680043.9994,C7,7,0,264,5,320,89,238,0,0.613176,0.811858,0.42,0.81,0.0


In [508]:
overall_infer_train = overall_infer[overall_infer['train'] == 1]
overall_infer_test = overall_infer[overall_infer['train'] == 0]

## Ensemble wtih various weights

#### Add model weight

In [509]:
two_stage_weight = [0.5,0.6,0.7,0.8,0.85,0.9,0.95]
for i in two_stage_weight:
    overall_infer_train[f'two_stage_{int(100*i)}'] = overall_infer_train['two_stage_pred']*i + overall_infer_train['effnet_pred']*(1-i)

#overall_infer_train['two_stage_pred_label'] = np.where(overall_infer_train['two_stage_pred']>=overall_infer_train['two_stage_threshold'],1,0)
#overall_infer_train['effnet_pred_label'] = np.where(overall_infer_train['effnet_pred']>=overall_infer_train['effnet_threshold'],1,0)
   
overall_infer_train

,StudyInstanceUID,prediction_type,cid,x0,x1,y0,y1,z0,z1,label,...,two_stage_threshold,effnet_threshold,train,two_stage_50,two_stage_60,two_stage_70,two_stage_80,two_stage_85,two_stage_90,two_stage_95
0,1.2.826.0.1.3680043.10032,C1,1,0,320,1,314,72,243,0,...,0.23,0.67,1.0,0.403362,0.345656,0.287950,0.230244,0.201391,0.172538,0.143685
1,1.2.826.0.1.3680043.10062,C1,1,0,349,1,316,2,262,0,...,0.23,0.67,1.0,0.212075,0.188022,0.163969,0.139917,0.127890,0.115864,0.103837
2,1.2.826.0.1.3680043.10136,C1,1,0,357,2,317,76,250,0,...,0.23,0.67,1.0,0.410053,0.353028,0.296002,0.238977,0.210464,0.181951,0.153438
3,1.2.826.0.1.3680043.10412,C1,1,0,684,2,216,69,230,0,...,0.23,0.67,1.0,0.403341,0.346839,0.290337,0.233835,0.205584,0.177333,0.149082
5,1.2.826.0.1.3680043.10697,C1,1,0,179,58,309,6,256,0,...,0.23,0.67,1.0,0.483487,0.431425,0.379362,0.327300,0.301269,0.275238,0.249207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3356,1.2.826.0.1.3680043.9774,C7,7,38,260,91,268,98,224,0,...,0.42,0.81,1.0,0.677815,0.648948,0.620081,0.591214,0.576780,0.562347,0.547913
3357,1.2.826.0.1.3680043.9809,C7,7,0,568,3,224,82,237,0,...,0.42,0.81,1.0,0.589657,0.549363,0.509069,0.468775,0.448628,0.428481,0.408334
3359,1.2.826.0.1.3680043.9940,C7,7,41,257,40,315,67,281,0,...,0.42,0.81,1.0,0.316154,0.340365,0.364575,0.388786,0.400892,0.412997,0.425102
3360,1.2.826.0.1.3680043.9994,C7,7,0,264,5,320,89,238,0,...,0.42,0.81,1.0,0.712517,0.692648,0.672780,0.652912,0.642978,0.633044,0.623110


## Evaluation

In [510]:
def define_eval_metrics(df, a, p):
    recall = np.round(recall_score(y_true=df[a], y_pred=df[p]) * 100, 2)
    precision = np.round(
        precision_score(y_true=df[a], y_pred=df[p]) * 100, 2
    )

    tn, fp, fn, tp = confusion_matrix(y_true=df[a], y_pred=df[p]).ravel()

    fpr = np.round(fp / (fp + tn) * 100, 2)
    fnr = np.round(fn / (fn + tp) * 100, 2)

    accuracy = np.round(
        accuracy_score(
            y_true=df[a],
            y_pred=df[p],
        )
        * 100
    )

    f1 = np.round(fbeta_score(y_true=df[a], y_pred=df[p], beta=1) * 100, 2)

    f05 = np.round(
        fbeta_score(y_true=df[a], y_pred=df[p], beta=0.5) * 100, 2
    )

    f2 = np.round(fbeta_score(y_true=df[a], y_pred=df[p], beta=2) * 100, 2)

    return [recall, precision, tn, fp, fn, tp, fpr, fnr, accuracy, f1, f05, f2]

In [511]:
def find_best_f1(df):

    two_stage_weight = [0.5, 0.6,0.7,0.8,0.85,0.9,0.95]

    label = ['two_stage_50_label','two_stage_60_label','two_stage_70_label',
         'two_stage_80_label','two_stage_85_label','two_stage_90_label','two_stage_95_label',
             'two_stage_pred_label','effnet_pred_label']
    
    for j in label:
        best_f1 = [0,0,0,0,0,0,0]
        best_threshold = [0.05,0.05,0.05,0.05,0.05,0.05,0.05]
        
        for curr_threshold in np.linspace(0.05, 0.50, 17):
            for i in two_stage_weight:
                df[f'two_stage_{int(100*i)}_label'] = np.where(df[f'two_stage_{int(100*i)}']>=curr_threshold,1,0)    
                
            df['two_stage_pred_label'] = np.where(df['two_stage_pred']>=curr_threshold,1,0)   
            df['effnet_pred_label'] = np.where(df['effnet_pred']>=curr_threshold,1,0)   
            
            #df.to_csv(f'df_{curr_threshold}_{j}.csv', index=False)
            
            metric_value_c1 = define_eval_metrics(df[df['prediction_type']=='C1'],'label', j)
            metric_value_c2 = define_eval_metrics(df[df['prediction_type']=='C2'],'label', j)
            metric_value_c3 = define_eval_metrics(df[df['prediction_type']=='C3'],'label', j)
            metric_value_c4 = define_eval_metrics(df[df['prediction_type']=='C4'],'label', j)
            metric_value_c5 = define_eval_metrics(df[df['prediction_type']=='C5'],'label', j)
            metric_value_c6 = define_eval_metrics(df[df['prediction_type']=='C6'],'label', j)
            metric_value_c7 = define_eval_metrics(df[df['prediction_type']=='C7'],'label', j)

            if metric_value_c1[9]> best_f1[0]:
                best_f1[0] = metric_value_c1[9]
                best_threshold[0] = curr_threshold
            if metric_value_c2[9]> best_f1[1]:
                best_f1[1] = metric_value_c2[9]
                best_threshold[1] = curr_threshold
            if metric_value_c3[9]> best_f1[2]:
                best_f1[2] = metric_value_c3[9]
                best_threshold[2] = curr_threshold
            if metric_value_c4[9]> best_f1[3]:
                best_f1[3] = metric_value_c4[9]
                best_threshold[3] = curr_threshold
            if metric_value_c5[9]> best_f1[4]:
                best_f1[4] = metric_value_c5[9]
                best_threshold[4] = curr_threshold
            if metric_value_c6[9]> best_f1[5]:
                best_f1[5] = metric_value_c6[9]
                best_threshold[5] = curr_threshold
            if metric_value_c7[9]> best_f1[6]:
                best_f1[6] = metric_value_c7[9]
                best_threshold[6] = curr_threshold

        print(j+'/n')
        print(pd.DataFrame(dict(vertebrae=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'],best_f1=best_f1,thresdhold=best_threshold)))

#### Get best thredhold for each ensemble model yields best F2

In [512]:
find_best_f1(overall_infer_train)

two_stage_50_label/n
  vertebrae  best_f1  thresdhold
0        C1    34.41    0.471875
1        C2    43.84    0.443750
2        C3     9.68    0.303125
3        C4    15.52    0.275000
4        C5    14.40    0.303125
5        C6    27.17    0.275000
6        C7    34.16    0.246875
two_stage_60_label/n
  vertebrae  best_f1  thresdhold
0        C1    37.29    0.500000
1        C2    44.30    0.443750
2        C3    10.96    0.275000
3        C4    14.93    0.246875
4        C5    14.53    0.275000
5        C6    27.74    0.359375
6        C7    34.23    0.275000
two_stage_70_label/n
  vertebrae  best_f1  thresdhold
0        C1    40.68    0.471875
1        C2    45.80    0.471875
2        C3    11.59    0.275000
3        C4    14.29    0.218750
4        C5    14.69    0.246875
5        C6    27.06    0.359375
6        C7    34.23    0.275000
two_stage_80_label/n
  vertebrae  best_f1  thresdhold
0        C1    44.07    0.443750
1        C2    45.26    0.471875
2        C3    13.11    0

#### Get the predicted label of each ensemble model based on best classification threshold

In [513]:
two_stage_50_thresdhold = pd.DataFrame(dict(prediction_type=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'], 
                                            two_stage_50_thresdhold= [0.47, 0.44, 0.3,0.28,0.30,0.28,0.25]))
two_stage_60_thresdhold = pd.DataFrame(dict(prediction_type=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'], 
                                            two_stage_60_thresdhold= [0.5, 0.44, 0.28,0.25,0.28,0.36,0.28]))
two_stage_70_thresdhold = pd.DataFrame(dict(prediction_type=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'], 
                                            two_stage_70_thresdhold= [0.47, 0.47, 0.28,0.22,0.25,0.36,0.28]))
two_stage_80_thresdhold = pd.DataFrame(dict(prediction_type=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'], 
                                            two_stage_80_thresdhold= [0.44, 0.47, 0.30,0.19,0.33,0.36,0.28]))
two_stage_85_thresdhold = pd.DataFrame(dict(prediction_type=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'], 
                                            two_stage_85_thresdhold= [0.5, 0.47, 0.30,0.19,0.36,0.39,0.33]))
two_stage_90_thresdhold = pd.DataFrame(dict(prediction_type=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'], 
                                            two_stage_90_thresdhold= [0.5, 0.47, 0.30,0.16,0.36,0.39,0.47]))
two_stage_95_thresdhold = pd.DataFrame(dict(prediction_type=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'], 
                                            two_stage_95_thresdhold= [0.47, 0.47, 0.30,0.16,0.33,0.39,0.42]))
two_stage_thresdhold = pd.DataFrame(dict(prediction_type=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'], 
                                            two_stage_thresdhold= [0.47, 0.47, 0.30,0.19,0.30,0.5,0.42]))
effnet_thresdhold = pd.DataFrame(dict(prediction_type=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'], 
                                            effnet_thresdhold= [0.33, 0.39, 0.13,0.28,0.44,0.05,0.19]))

overall_infer_train = overall_infer_train.merge(two_stage_50_thresdhold, how='inner',
                                left_on = 'prediction_type', right_on = 'prediction_type')
overall_infer_train = overall_infer_train.merge(two_stage_60_thresdhold, how='inner',
                                left_on = 'prediction_type', right_on = 'prediction_type')
overall_infer_train = overall_infer_train.merge(two_stage_70_thresdhold, how='inner',
                                left_on = 'prediction_type', right_on = 'prediction_type')
overall_infer_train = overall_infer_train.merge(two_stage_80_thresdhold, how='inner',
                                left_on = 'prediction_type', right_on = 'prediction_type')
overall_infer_train = overall_infer_train.merge(two_stage_85_thresdhold, how='inner',
                                left_on = 'prediction_type', right_on = 'prediction_type')
overall_infer_train = overall_infer_train.merge(two_stage_90_thresdhold, how='inner',
                                left_on = 'prediction_type', right_on = 'prediction_type')
overall_infer_train = overall_infer_train.merge(two_stage_95_thresdhold, how='inner',
                                left_on = 'prediction_type', right_on = 'prediction_type')
overall_infer_train = overall_infer_train.merge(two_stage_thresdhold, how='inner',
                                left_on = 'prediction_type', right_on = 'prediction_type')
overall_infer_train = overall_infer_train.merge(effnet_thresdhold, how='inner',
                                left_on = 'prediction_type', right_on = 'prediction_type')
overall_infer_train

,StudyInstanceUID,prediction_type,cid,x0,x1,y0,y1,z0,z1,label,...,effnet_pred_label,two_stage_50_thresdhold,two_stage_60_thresdhold,two_stage_70_thresdhold,two_stage_80_thresdhold,two_stage_85_thresdhold,two_stage_90_thresdhold,two_stage_95_thresdhold,two_stage_thresdhold,effnet_thresdhold
0,1.2.826.0.1.3680043.10032,C1,1,0,320,1,314,72,243,0,...,1,0.47,0.50,0.47,0.44,0.50,0.50,0.47,0.47,0.33
1,1.2.826.0.1.3680043.10062,C1,1,0,349,1,316,2,262,0,...,0,0.47,0.50,0.47,0.44,0.50,0.50,0.47,0.47,0.33
2,1.2.826.0.1.3680043.10136,C1,1,0,357,2,317,76,250,0,...,1,0.47,0.50,0.47,0.44,0.50,0.50,0.47,0.47,0.33
3,1.2.826.0.1.3680043.10412,C1,1,0,684,2,216,69,230,0,...,1,0.47,0.50,0.47,0.44,0.50,0.50,0.47,0.47,0.33
4,1.2.826.0.1.3680043.10697,C1,1,0,179,58,309,6,256,0,...,1,0.47,0.50,0.47,0.44,0.50,0.50,0.47,0.47,0.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2680,1.2.826.0.1.3680043.9774,C7,7,38,260,91,268,98,224,0,...,1,0.25,0.28,0.28,0.28,0.33,0.47,0.42,0.42,0.19
2681,1.2.826.0.1.3680043.9809,C7,7,0,568,3,224,82,237,0,...,1,0.25,0.28,0.28,0.28,0.33,0.47,0.42,0.42,0.19
2682,1.2.826.0.1.3680043.9940,C7,7,41,257,40,315,67,281,0,...,0,0.25,0.28,0.28,0.28,0.33,0.47,0.42,0.42,0.19
2683,1.2.826.0.1.3680043.9994,C7,7,0,264,5,320,89,238,0,...,1,0.25,0.28,0.28,0.28,0.33,0.47,0.42,0.42,0.19


In [514]:
two_stage_weight = [0.5,0.6,0.7,0.8,0.85,0.9,0.95]
for i in two_stage_weight:
    overall_infer_train[f'two_stage_{int(100*i)}_label'] = np.where(overall_infer_train[f'two_stage_{int(100*i)}']>=overall_infer_train[f'two_stage_{int(100*i)}_thresdhold'],1,0) 
    
overall_infer_train['two_stage_pred_label'] = np.where(overall_infer_train['two_stage_pred']>=overall_infer_train['two_stage_thresdhold'],1,0)
overall_infer_train['effnet_pred_label'] = np.where(overall_infer_train['effnet_pred']>=overall_infer_train['effnet_thresdhold'],1,0)

overall_infer_train

,StudyInstanceUID,prediction_type,cid,x0,x1,y0,y1,z0,z1,label,...,effnet_pred_label,two_stage_50_thresdhold,two_stage_60_thresdhold,two_stage_70_thresdhold,two_stage_80_thresdhold,two_stage_85_thresdhold,two_stage_90_thresdhold,two_stage_95_thresdhold,two_stage_thresdhold,effnet_thresdhold
0,1.2.826.0.1.3680043.10032,C1,1,0,320,1,314,72,243,0,...,1,0.47,0.50,0.47,0.44,0.50,0.50,0.47,0.47,0.33
1,1.2.826.0.1.3680043.10062,C1,1,0,349,1,316,2,262,0,...,1,0.47,0.50,0.47,0.44,0.50,0.50,0.47,0.47,0.33
2,1.2.826.0.1.3680043.10136,C1,1,0,357,2,317,76,250,0,...,1,0.47,0.50,0.47,0.44,0.50,0.50,0.47,0.47,0.33
3,1.2.826.0.1.3680043.10412,C1,1,0,684,2,216,69,230,0,...,1,0.47,0.50,0.47,0.44,0.50,0.50,0.47,0.47,0.33
4,1.2.826.0.1.3680043.10697,C1,1,0,179,58,309,6,256,0,...,1,0.47,0.50,0.47,0.44,0.50,0.50,0.47,0.47,0.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2680,1.2.826.0.1.3680043.9774,C7,7,38,260,91,268,98,224,0,...,1,0.25,0.28,0.28,0.28,0.33,0.47,0.42,0.42,0.19
2681,1.2.826.0.1.3680043.9809,C7,7,0,568,3,224,82,237,0,...,1,0.25,0.28,0.28,0.28,0.33,0.47,0.42,0.42,0.19
2682,1.2.826.0.1.3680043.9940,C7,7,41,257,40,315,67,281,0,...,1,0.25,0.28,0.28,0.28,0.33,0.47,0.42,0.42,0.19
2683,1.2.826.0.1.3680043.9994,C7,7,0,264,5,320,89,238,0,...,1,0.25,0.28,0.28,0.28,0.33,0.47,0.42,0.42,0.19


#### Get patient over label

In [515]:
overall_infer_train_overall = overall_infer_train.groupby('StudyInstanceUID')[['label','two_stage_pred_label','effnet_pred_label',
                                                       'two_stage_50_label','two_stage_60_label',
                                                       'two_stage_70_label','two_stage_80_label','two_stage_85_label',
                                                       'two_stage_90_label','two_stage_95_label']].sum().reset_index()
overall_infer_train_overall['label'] =  np.where(overall_infer_train_overall['label']>0,1,0)
overall_infer_train_overall['two_stage_pred_label'] =  np.where(overall_infer_train_overall['two_stage_pred_label']>0,1,0)
overall_infer_train_overall['effnet_pred_label'] =  np.where(overall_infer_train_overall['effnet_pred_label']>0,1,0)
overall_infer_train_overall['two_stage_50_label'] =  np.where(overall_infer_train_overall['two_stage_50_label']>0,1,0)
overall_infer_train_overall['two_stage_60_label'] =  np.where(overall_infer_train_overall['two_stage_60_label']>0,1,0)
overall_infer_train_overall['two_stage_70_label'] =  np.where(overall_infer_train_overall['two_stage_70_label']>0,1,0)
overall_infer_train_overall['two_stage_80_label'] =  np.where(overall_infer_train_overall['two_stage_80_label']>0,1,0)
overall_infer_train_overall['two_stage_85_label'] =  np.where(overall_infer_train_overall['two_stage_85_label']>0,1,0)
overall_infer_train_overall['two_stage_90_label'] =  np.where(overall_infer_train_overall['two_stage_90_label']>0,1,0)
overall_infer_train_overall['two_stage_95_label'] =  np.where(overall_infer_train_overall['two_stage_95_label']>0,1,0)
overall_infer_train_overall['prediction_type'] = 'patient_overall'
overall_infer_train_overall

,StudyInstanceUID,label,two_stage_pred_label,effnet_pred_label,two_stage_50_label,two_stage_60_label,two_stage_70_label,two_stage_80_label,two_stage_85_label,two_stage_90_label,two_stage_95_label,prediction_type
0,1.2.826.0.1.3680043.10032,0,1,1,1,1,1,1,1,1,1,patient_overall
1,1.2.826.0.1.3680043.10062,0,1,1,1,1,1,1,1,0,1,patient_overall
2,1.2.826.0.1.3680043.10136,0,1,1,1,1,1,1,1,1,1,patient_overall
3,1.2.826.0.1.3680043.10412,0,0,1,1,1,1,1,1,1,1,patient_overall
4,1.2.826.0.1.3680043.10697,1,1,1,1,1,1,1,1,1,1,patient_overall
...,...,...,...,...,...,...,...,...,...,...,...,...
395,1.2.826.0.1.3680043.9774,1,1,1,1,1,1,1,1,1,1,patient_overall
396,1.2.826.0.1.3680043.9809,0,0,1,1,1,1,1,1,0,0,patient_overall
397,1.2.826.0.1.3680043.9940,1,1,1,1,1,1,1,1,1,1,patient_overall
398,1.2.826.0.1.3680043.9994,0,1,1,1,1,1,1,1,1,1,patient_overall


In [516]:
overall_infer_train_full = pd.concat([overall_infer_train_overall, 
                                      overall_infer_train[['StudyInstanceUID', 'label','two_stage_pred_label','effnet_pred_label',
                                                        'two_stage_50_label','two_stage_60_label',
                                                       'two_stage_70_label','two_stage_80_label','two_stage_85_label',
                                                       'two_stage_90_label','two_stage_95_label','prediction_type']]])
overall_infer_train_full

,StudyInstanceUID,label,two_stage_pred_label,effnet_pred_label,two_stage_50_label,two_stage_60_label,two_stage_70_label,two_stage_80_label,two_stage_85_label,two_stage_90_label,two_stage_95_label,prediction_type
0,1.2.826.0.1.3680043.10032,0,1,1,1,1,1,1,1,1,1,patient_overall
1,1.2.826.0.1.3680043.10062,0,1,1,1,1,1,1,1,0,1,patient_overall
2,1.2.826.0.1.3680043.10136,0,1,1,1,1,1,1,1,1,1,patient_overall
3,1.2.826.0.1.3680043.10412,0,0,1,1,1,1,1,1,1,1,patient_overall
4,1.2.826.0.1.3680043.10697,1,1,1,1,1,1,1,1,1,1,patient_overall
...,...,...,...,...,...,...,...,...,...,...,...,...
2680,1.2.826.0.1.3680043.9774,0,1,1,1,1,1,1,1,1,1,C7
2681,1.2.826.0.1.3680043.9809,0,0,1,1,1,1,1,1,0,0,C7
2682,1.2.826.0.1.3680043.9940,0,1,1,1,1,1,1,1,0,1,C7
2683,1.2.826.0.1.3680043.9994,0,1,1,1,1,1,1,1,1,1,C7


#### Evaluation of best weight

In [517]:
model_type = []
metric = []
patient_overall = []
c1 = []
c2 = []
c3 = []
c4 = []
c5 = []
c6 = []
c7 = []

vertebrae = ['patient_overall', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']
label = ['two_stage_pred_label','effnet_pred_label','two_stage_50_label','two_stage_60_label','two_stage_70_label',
         'two_stage_80_label','two_stage_85_label','two_stage_90_label','two_stage_95_label']
metric_name = ['recall', 'precision', 'tn', 'fp', 'fn', 'tp', 'fpr', 'fnr', 'accuracy', 'f1', 'f05', 'f2']
    
for j in label:
        
    metric_value_overall = define_eval_metrics(overall_infer_train_full[overall_infer_train_full['prediction_type']=='patient_overall'],'label', j)
    metric_value_c1 = define_eval_metrics(overall_infer_train_full[overall_infer_train_full['prediction_type']=='C1'],'label', j)
    metric_value_c2 = define_eval_metrics(overall_infer_train_full[overall_infer_train_full['prediction_type']=='C2'],'label', j)
    metric_value_c3 = define_eval_metrics(overall_infer_train_full[overall_infer_train_full['prediction_type']=='C3'],'label', j)
    metric_value_c4 = define_eval_metrics(overall_infer_train_full[overall_infer_train_full['prediction_type']=='C4'],'label', j)
    metric_value_c5 = define_eval_metrics(overall_infer_train_full[overall_infer_train_full['prediction_type']=='C5'],'label', j)
    metric_value_c6 = define_eval_metrics(overall_infer_train_full[overall_infer_train_full['prediction_type']=='C6'],'label', j)
    metric_value_c7 = define_eval_metrics(overall_infer_train_full[overall_infer_train_full['prediction_type']=='C7'],'label', j)
       
    for m in range(12):
        model_type.append(j)
        metric.append(metric_name[m])
        patient_overall.append(metric_value_overall[m])
        c1.append(metric_value_c1[m])
        c2.append(metric_value_c2[m])
        c3.append(metric_value_c3[m])
        c4.append(metric_value_c4[m])
        c5.append(metric_value_c5[m])
        c6.append(metric_value_c6[m])
        c7.append(metric_value_c7[m])


df_metric = pd.DataFrame(dict(model_type=model_type,metric=metric,patient_overall=patient_overall,
                             c1=c1,c2=c2,c3=c3,c4=c4,c5=c5,c6=c6,c7=c7))
df_metric

,model_type,metric,patient_overall,c1,c2,c3,c4,c5,c6,c7
0,two_stage_pred_label,recall,91.11,41.38,62.26,28.57,52.63,46.43,32.00,90.79
1,two_stage_pred_label,precision,49.10,52.17,34.02,8.89,9.71,12.87,24.24,22.85
2,two_stage_pred_label,tn,50.00,347.00,280.00,342.00,279.00,252.00,270.00,66.00
3,two_stage_pred_label,fp,170.00,11.00,64.00,41.00,93.00,88.00,50.00,233.00
4,two_stage_pred_label,fn,16.00,17.00,20.00,10.00,9.00,15.00,34.00,7.00
...,...,...,...,...,...,...,...,...,...,...
103,two_stage_95_label,fnr,6.67,58.62,37.74,71.43,15.79,57.14,42.00,9.21
104,two_stage_95_label,accuracy,52.00,93.00,79.00,87.00,60.00,73.00,61.00,33.00
105,two_stage_95_label,f1,63.52,45.28,44.30,13.56,17.02,19.67,28.57,35.57
106,two_stage_95_label,f05,53.30,48.00,37.76,10.31,11.51,14.85,21.90,26.06


In [518]:
df_metric.to_csv('df_metric_f1.csv', index=False)

## Best weight According to df_metric

In [528]:
best_weight = pd.DataFrame(dict(prediction_type = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'],
                                two_stage_weight = [1,0.7,0.95,0.8,1,0.8,1],
                                ensemble_threshold = [0.47,0.47,0.3,0.19,0.3,0.36,0.42]))
best_weight

,prediction_type,two_stage_weight,ensemble_threshold
0,C1,1.00,0.47
1,C2,0.70,0.47
2,C3,0.95,0.30
3,C4,0.80,0.19
4,C5,1.00,0.30
5,C6,0.80,0.36
6,C7,1.00,0.42


## Final Ensemble Model

In [519]:
def ensemble(df):
    best_weight = pd.DataFrame(dict(prediction_type = [ 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'],
                                    two_stage_weight = [1,0.7,0.95,0.8,1,0.8,1],
                                   ensemble_threshold =[0.47,0.47,0.3,0.19,0.3,0.36,0.42]))
   
    threshold = pd.DataFrame({'prediction_type': ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'],
              'two_stage_threshold': [0.47, 0.47, 0.30,0.19,0.30,0.5,0.42],
              'effnet_threshold': [0.33, 0.39, 0.13,0.28,0.44,0.05,0.19],
                 })
    df = df.drop(columns=['two_stage_threshold', 'effnet_threshold'])
    df = df.merge(threshold, how='inner', left_on = 'prediction_type', right_on = 'prediction_type')
    
   # print(df)
    
    df['two_stage_pred_label'] = np.where(df['two_stage_pred']>=df['two_stage_threshold'],1,0)
    df['effnet_pred_label'] = np.where(df['effnet_pred']>=df['effnet_threshold'],1,0)
   
    
    df = df.merge(best_weight, how='inner',
                                left_on = 'prediction_type', right_on = 'prediction_type')

    df['ensemble_pred'] = df['two_stage_pred']*df['two_stage_weight'] + df['effnet_pred']*(1-df['two_stage_weight'])

    df['ensemble_label'] = np.where(df['ensemble_pred']>=df['ensemble_threshold'],1,0)
    
    df_overall = df.groupby('StudyInstanceUID')[['label','two_stage_pred_label','effnet_pred_label','ensemble_label']].sum().reset_index()
    df_overall['label'] =  np.where(df_overall['label']>0,1,0)
    df_overall['two_stage_pred_label'] =  np.where(df_overall['two_stage_pred_label']>0,1,0)
    df_overall['effnet_pred_label'] =  np.where(df_overall['effnet_pred_label']>0,1,0)
    df_overall['ensemble_label'] =  np.where(df_overall['ensemble_label']>0,1,0)
    df_overall['prediction_type'] = 'patient_overall'

    df_full = pd.concat([df_overall, 
                         df[['StudyInstanceUID', 'label','two_stage_pred_label','effnet_pred_label','ensemble_label','prediction_type']]])

    return df_full

In [520]:
overall_infer = pd.read_csv('overall_infer.csv')
overall_infer_train_final = overall_infer[overall_infer['train'] == 1]
overall_infer_test_final = overall_infer[overall_infer['train'] == 0]

#### Result

In [521]:
ensemble_train = ensemble(overall_infer_train_final)
ensemble_train.to_csv('ensemble_train_f1.csv', index=False)

In [522]:
ensemble_test = ensemble(overall_infer_test_final)
ensemble_test.to_csv('ensemble_test_f1.csv', index=False)

In [523]:
ensemble_overall = ensemble(overall_infer)
ensemble_overall.to_csv('ensemble_overall_f1.csv', index=False)

In [524]:
def ensemble_eval(df):
    model_type = []
    metric = []
    patient_overall = []
    c1 = []
    c2 = []
    c3 = []
    c4 = []
    c5 = []
    c6 = []
    c7 = []

    vertebrae = ['patient_overall', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']
    label = ['two_stage_pred_label','effnet_pred_label','ensemble_label']
    metric_name = ['recall', 'precision', 'tn', 'fp', 'fn', 'tp', 'fpr', 'fnr', 'accuracy', 'f1', 'f05', 'f2']

    for j in label:

        metric_value_overall = define_eval_metrics(df[df['prediction_type']=='patient_overall'],'label', j)
        metric_value_c1 = define_eval_metrics(df[df['prediction_type']=='C1'],'label', j)
        metric_value_c2 = define_eval_metrics(df[df['prediction_type']=='C2'],'label', j)
        metric_value_c3 = define_eval_metrics(df[df['prediction_type']=='C3'],'label', j)
        metric_value_c4 = define_eval_metrics(df[df['prediction_type']=='C4'],'label', j)
        metric_value_c5 = define_eval_metrics(df[df['prediction_type']=='C5'],'label', j)
        metric_value_c6 = define_eval_metrics(df[df['prediction_type']=='C6'],'label', j)
        metric_value_c7 = define_eval_metrics(df[df['prediction_type']=='C7'],'label', j)

        for m in range(12):
            model_type.append(j)
            metric.append(metric_name[m])
            patient_overall.append(metric_value_overall[m])
            c1.append(metric_value_c1[m])
            c2.append(metric_value_c2[m])
            c3.append(metric_value_c3[m])
            c4.append(metric_value_c4[m])
            c5.append(metric_value_c5[m])
            c6.append(metric_value_c6[m])
            c7.append(metric_value_c7[m])


    df_metric = pd.DataFrame(dict(model_type=model_type,metric=metric,patient_overall=patient_overall,
                                 c1=c1,c2=c2,c3=c3,c4=c4,c5=c5,c6=c6,c7=c7))
    return df_metric

In [525]:
eval_train = ensemble_eval(ensemble_train)
eval_train.to_csv('ensemble_eval_train_f1.csv', index=False)
eval_train

,model_type,metric,patient_overall,c1,c2,c3,c4,c5,c6,c7
0,two_stage_pred_label,recall,91.11,41.38,62.26,28.57,52.63,46.43,32.00,90.79
1,two_stage_pred_label,precision,49.10,52.17,34.02,8.89,9.71,12.87,24.24,22.85
2,two_stage_pred_label,tn,50.00,347.00,280.00,342.00,279.00,252.00,270.00,66.00
3,two_stage_pred_label,fp,170.00,11.00,64.00,41.00,93.00,88.00,50.00,233.00
4,two_stage_pred_label,fn,16.00,17.00,20.00,10.00,9.00,15.00,34.00,7.00
5,two_stage_pred_label,tp,164.00,12.00,33.00,4.00,10.00,13.00,16.00,69.00
6,two_stage_pred_label,fpr,77.27,3.07,18.60,10.70,25.00,25.88,15.62,77.93
7,two_stage_pred_label,fnr,8.89,58.62,37.74,71.43,47.37,53.57,68.00,9.21
8,two_stage_pred_label,accuracy,54.00,93.00,79.00,87.00,74.00,72.00,77.00,36.00
9,two_stage_pred_label,f1,63.81,46.15,44.00,13.56,16.39,20.16,27.59,36.51


In [526]:
eval_test = ensemble_eval(ensemble_test)
eval_test.to_csv('ensemble_eval_test_f1.csv', index=False)
eval_test

,model_type,metric,patient_overall,c1,c2,c3,c4,c5,c6,c7
0,two_stage_pred_label,recall,91.11,16.67,80.00,50.00,25.00,33.33,40.00,88.46
1,two_stage_pred_label,precision,45.05,20.00,23.53,7.14,2.78,5.71,19.05,28.40
2,two_stage_pred_label,tn,5.00,88.00,63.00,85.00,59.00,53.00,68.00,11.00
3,two_stage_pred_label,fp,50.00,4.00,26.00,13.00,35.00,33.00,17.00,58.00
4,two_stage_pred_label,fn,4.00,5.00,2.00,1.00,3.00,4.00,6.00,3.00
5,two_stage_pred_label,tp,41.00,1.00,8.00,1.00,1.00,2.00,4.00,23.00
6,two_stage_pred_label,fpr,90.91,4.35,29.21,13.27,37.23,38.37,20.00,84.06
7,two_stage_pred_label,fnr,8.89,83.33,20.00,50.00,75.00,66.67,60.00,11.54
8,two_stage_pred_label,accuracy,46.00,91.00,72.00,86.00,61.00,60.00,76.00,36.00
9,two_stage_pred_label,f1,60.29,18.18,36.36,12.50,5.00,9.76,25.81,42.99


In [527]:
eval_overall = ensemble_eval(ensemble_overall)
eval_overall.to_csv('ensemble_eval_overall_f1.csv', index=False)
eval_overall

,model_type,metric,patient_overall,c1,c2,c3,c4,c5,c6,c7
0,two_stage_pred_label,recall,91.11,37.14,65.08,31.25,47.83,44.12,33.33,90.20
1,two_stage_pred_label,precision,48.24,46.43,31.30,8.47,7.91,11.03,22.99,24.02
2,two_stage_pred_label,tn,55.00,435.00,343.00,427.00,338.00,305.00,338.00,77.00
3,two_stage_pred_label,fp,220.00,15.00,90.00,54.00,128.00,121.00,67.00,291.00
4,two_stage_pred_label,fn,20.00,22.00,22.00,11.00,12.00,19.00,40.00,10.00
5,two_stage_pred_label,tp,205.00,13.00,41.00,5.00,11.00,15.00,20.00,92.00
6,two_stage_pred_label,fpr,80.00,3.33,20.79,11.23,27.47,28.40,16.54,79.08
7,two_stage_pred_label,fnr,8.89,62.86,34.92,68.75,52.17,55.88,66.67,9.80
8,two_stage_pred_label,accuracy,52.00,92.00,77.00,87.00,71.00,70.00,77.00,36.00
9,two_stage_pred_label,f1,63.08,41.27,42.27,13.33,13.58,17.65,27.21,37.94
